<a href="https://colab.research.google.com/github/ShotaroBaba/Machine_Learning_Practice/blob/master/Image_processing_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys

import tensorflow as tf
import numpy as np
import os
import urllib.request
import shutil
from PIL import Image
import json
from imageio import imread

from matplotlib import pyplot as plt
from tensorflow import keras

In [0]:
if not os.path.isdir("results"):
    os.mkdir("results")


In [103]:
!if [ ! -f cifar-100-python.tar.gz ] ; then wget http://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz ; fi
!tar -xvzf cifar-100-python.tar.gz

cifar-100-python/
cifar-100-python/file.txt~
cifar-100-python/train
cifar-100-python/test
cifar-100-python/meta


In [104]:
!ls

cifar-100-python  cifar-100-python.tar.gz  results  sample_data  saved_model.h5


In [0]:
# The function which unpickle train and test files
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [0]:
# Unpickle files
train = unpickle('cifar-100-python/train')
test = unpickle('cifar-100-python/test')

In [107]:
print(train.keys())
print(test.keys())

dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])
dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])


In [108]:
# Print a number of train data
print(f'The number of train data: {len(train[b"data"])}')
print(f'The number of test data: {len(test[b"data"])}')

print(f'The number of fine train label: {len(train[b"fine_labels"])}')
print(f'The number of fine test label: {len(test[b"fine_labels"])}')

print(f'The number of coarse train data: {len(train[b"coarse_labels"])}')
print(f'The number of coarse test data: {len(test[b"coarse_labels"])}')



The number of train data: 50000
The number of test data: 10000
The number of fine train label: 50000
The number of fine test label: 10000
The number of coarse train data: 50000
The number of coarse test data: 10000


In [109]:
# Print some of the contents in data:
print(f'The contents of train data: {train[b"data"][0]}')
print(f'The contents of test data: {test[b"data"][0]}')

print(f'The contents of fine train label: {train[b"fine_labels"][0]}')
print(f'The contents of fine test label: {test[b"fine_labels"][0]}')

print(f'The contents of coarse train data: {train[b"coarse_labels"][0]}')
print(f'The contents of coarse test data: {test[b"coarse_labels"][0]}')


The contents of train data: [255 255 255 ...  10  59  79]
The contents of test data: [199 196 195 ... 215 211 183]
The contents of fine train label: 19
The contents of fine test label: 49
The contents of coarse train data: 11
The contents of coarse test data: 10


In [110]:
# Use fine tune label for training data:
print(f'The contents of test data: {len(test[b"data"][0])}')


The contents of test data: 3072


In [0]:
# First, pre-process the data:
def process_data(data):
    return [[data[i], data[i+1024], data[i+2048]] for i in range(0,1024)]

processed_train_data = np.array([process_data(data) for data in train[b"data"]])
processed_test_data = np.array([process_data(data) for data in test[b"data"]])

In [0]:
processed_train_data = processed_train_data.reshape(50000,32,32,3)
processed_test_data = processed_test_data.reshape(10000,32,32,3)

In [0]:
train_labels = np.array(train[b"fine_labels"])
test_labels = np.array(test[b"fine_labels"])



In [114]:
print(len(np.unique(train_labels)))
print(len(np.unique(test_labels)))

100
100


In [0]:
processed_train_data, processed_test_data = processed_train_data / 255.0,  processed_test_data / 255.0

In [116]:
# Construct convolutional neural network first to enable image categorisation.
# Reduce the number of array due to memory shortage.
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.Conv2D(256, (3, 3), activation='relu'))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(100, activation='softmax'))

model.summary()

model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

history = model.fit(processed_train_data, train_labels, epochs=20, 
                    validation_data=(processed_test_data, test_labels))




Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 128)       0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 13, 13, 128)       147584    
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 11, 11, 256)       295168    
_________________________________________________________________
flatten_7 (Flatten)          (None, 30976)             0         
_________________________________________________________________
dense_14 (Dense)             (None, 1024)              31720448  
_________________________________________________________________
dense_15 (Dense)             (None, 100)              

In [0]:
model_path = "saved_model.h5"
model.save(model_path)

In [0]:
model = keras.models.load_model(model_path)

# Only take 3000 images in this case.
predicted_results = model.predict(processed_test_data[:100])
sampled_test_labels = test_labels[:100]

sample_test_images = processed_test_data[:100]

In [119]:
print(sample_test_images[0])

[[[0.78039216 0.84313725 0.97647059]
  [0.76862745 0.82745098 0.95686275]
  [0.76470588 0.82352941 0.95294118]
  ...
  [0.84705882 0.90588235 0.98039216]
  [0.85098039 0.90588235 0.98039216]
  [0.87843137 0.91764706 0.98823529]]

 [[0.77254902 0.82352941 0.9372549 ]
  [0.76470588 0.81568627 0.93333333]
  [0.76470588 0.82352941 0.94117647]
  ...
  [0.90588235 0.95294118 0.98039216]
  [0.91372549 0.95294118 0.98039216]
  [0.94509804 0.96078431 0.99215686]]

 [[0.87058824 0.88627451 0.96470588]
  [0.83529412 0.8627451  0.94901961]
  [0.81960784 0.85882353 0.95294118]
  ...
  [0.95294118 0.98039216 0.98431373]
  [0.95686275 0.97647059 0.98431373]
  [0.98039216 0.98039216 0.99215686]]

 ...

 [[0.28235294 0.28627451 0.38823529]
  [0.27843137 0.29019608 0.4       ]
  [0.29019608 0.30588235 0.42352941]
  ...
  [0.8627451  0.81568627 0.85098039]
  [0.71764706 0.65882353 0.70980392]
  [0.60784314 0.55294118 0.58823529]]

 [[0.28235294 0.29411765 0.40784314]
  [0.29803922 0.31764706 0.43529412]


In [120]:
# Save the results of the image categorisation.
for i, img_data in enumerate(sample_test_images):
    predicted = predicted_results[i].argmax()
    truth =  sampled_test_labels[i]
    if predicted == truth:
        plt.savefig("results/right_predicted_results_{0}_{1}_{2}.png".format(predicted, truth, i))
    else:
        plt.savefig("results/wrong_predicted_results_{0}_{1}_{2}.png".format(predicted, truth, i))

<Figure size 432x288 with 0 Axes>

In [0]:
# Show the predicted results.
!ls results

In [0]:
# Reference:
# Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009.